In [1]:
import jax
from jax import numpy as jp
from matplotlib.lines import Line2D
from matplotlib.patches import Circle
import matplotlib.pyplot as plt

import brax

In [2]:
print(brax.__version__)
import IPython
from IPython.display import HTML, clear_output
print(IPython.__version__)
from brax.io import model
from brax.io import json
from brax.io import html

0.9.0
8.13.2


In [3]:
from brax.io import mjcf

ball = mjcf.loads(
    """<mujoco>
         <option timestep="0.005"/>
         <worldbody>
           <body pos="0 0 3">
             <joint type="free"/>
             <geom size="0.5 1 1" type="sphere"/>
           </body>
           <geom size="40 40 40" type="plane"/>
         </worldbody>
       </mujoco>
  """)

In [7]:
print(ball.init_q)

[0. 0. 3. 1. 0. 0. 0.]


In [23]:
#@title { run: "auto"}
from brax.positional import pipeline

elasticity = 0.85 #@param { type:"slider", min: 0.5, max: 1.0, step:0.05 }
ball_velocity = 1 #@param { type:"slider", min:-5, max:5, step: 0.5 }

# change the material elasticity of the ball and the plane
geoms = [g.replace(elasticity=jp.array([elasticity])) for g in ball.geoms]
ball = ball.replace(geoms=geoms)

# provide an initial velocity to the ball
qd = jp.zeros(ball.qd_size())
state = jax.jit(pipeline.init)(ball, ball.init_q, qd)

html.save('ball.html', ball, [state])